##KNN

In [20]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
#sl:satisfaction_level-False:MinMaxScaler,True:StandardScaler
#le:last_evaluation-False:MinMaxScaler,True:StandardScaler
#npr:number_project-False;MinMaxScaler,True:StandardScaler
#amh:average_monthly-hours-False;MinMaxScaler,True:StandardScaler
#tsc:time_spend_company-False;MinMaxScaler,True:StandardScaler
#wa:work_accident-False;MinMaxScaler,True:StandardScaler
#pl5:promotion_last_5years-False;MinMaxScaler,True:StandardScaler
#dp:department-False:LabelEncoding,True:OneHotEncoding
#slr；salary-False:LabelEncoding,True:OneHotEncoding
#lower_d指定是否需要降维，如需要默认降到一维
def hr_preprocessing(sl=False,le=False,npr=False,amh=False,tsc=False,wa=False,pl5=False,dp=False,slr=False,lower_d=False,ld_n=1):
    df=pd.read_csv('./data/HR.csv')
   
    #1.清洗数据(去除异常值、抽样)
    df=df.dropna(subset=['satisfaction_level','last_evaluation'])
    df=df[df['satisfaction_level']<=1][df['salary']!='nme']
     #2。得到标注
    label = df['left']
    df=df.drop('left',axis=1)#axis=1指定以列删除
    #3.特征选择
    #4.特征处理  
    scaler_lst=[sl,le,npr,amh,tsc,wa,pl5]
    column_lst=['satisfaction_level','last_evaluation','number_project','average_monthly_hours','time_spend_company','Work_accident','promotion_last_5years']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=\
                MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]#二维格式
        else:
            df[column_lst[i]]=\
                StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
      
    scaler_lst=[dp,slr]
    column_lst=['department','salary']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i]=='salary':
                df[column_lst[i]]=[map_salary(s) for s in df['salary'].values]
            else:
                df[column_lst[i]]=LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df=pd.get_dummies(df,columns=[column_lst[i]])
    if lower_d:
        return PCA(n_components=ld_n).fit_transform(df,values).label#PVA降维，可不使用标注
    return df,label
#使salary返回对应的等级而不是012
d=dict([('low',0),('medium',1),('high',2)])
def map_salary(s):
    return d.get(s,0)
#建模
def hr_modeling(features,label):
    from sklearn.model_selection import train_test_split  #切分训练集测试集
    f_v=features.values
    l_v=label.values
    X_tt,X_validation,Y_tt,Y_validation=train_test_split(f_v,l_v,test_size=0.2)   #获得验证集占20%
    X_train,X_test,Y_train,Y_test=train_test_split(X_tt,Y_tt,test_size=0.25)  #训练集、测试集
    print(len(X_train),len(X_validation),len(X_test))#得到6：2：2
    #KNN
    from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier
    #knn_elf=KNeighborsClassifier(n_neighbors=5)
    knn_elf=KNeighborsClassifier(n_neighbors=3)
    knn_elf_n5=KNeighborsClassifier(n_neighbors=5)
    knn_elf.fit(X_train,Y_train)  #拟合获得模型
    knn_elf_n5.fit(X_train,Y_train)
   # Y_pred =knn_elf.predict(X_validation)  #预测验证集，获得验证值
    Y_pred_n5=knn_elf.predict(X_validation)
    from sklearn.metrics import accuracy_score,recall_score,f1_score#F一
    #预测值与实际值比较获得准确度，召回率，F值  
    #训练集
    print('Train：')
    Y_pred=knn_elf.predict(X_train)
    print('ACC:',accuracy_score(Y_train,Y_pred))
    print('REC:',recall_score(Y_train,Y_pred))
    print('F-SCORE:',f1_score(Y_train,Y_pred))#综合反映
    print('Validation：')
    Y_pred =knn_elf.predict(X_validation)
    print('ACC:',accuracy_score(Y_validation,Y_pred))
    print('REC:',recall_score(Y_validation,Y_pred))
    print('F-SCORE:',f1_score(Y_validation,Y_pred))
    print('ACC:',accuracy_score(Y_validation,Y_pred_n5))
    print('REC:',recall_score(Y_validation,Y_pred_n5))
    print('F-SCORE:',f1_score(Y_validation,Y_pred_n5))
    #通过测试集对比
    print('Test:')
    Y_pred=knn_elf.predict(X_test)
    print('ACC:',accuracy_score(Y_test,Y_pred))
    print('REC:',recall_score(Y_test,Y_pred))
    print('F-SCORE:',f1_score(Y_test,Y_pred))
    #存储训练出来的模型
    from sklearn.externals import joblib
    joblib.dump(knn_elf,'knn_elf')#后边参数为自己为模型起的名字
    #使用模型
    knn_elf=joblib.load('knn_elf')
    print('Test2:')
    Y_pred=knn_elf.predict(X_test)
    print('ACC:',accuracy_score(Y_test,Y_pred))
    print('REC:',recall_score(Y_test,Y_pred))
    print('F-SCORE:',f1_score(Y_test,Y_pred))
def main():
    #features,label=hr_preprocessing(sl=True,le=True,npr=True,amh=True,tsc=True,wa=True,pl5=True,dp=True,slr=False,lower_d=False)
    #features,label=hr_preprocessing(sl=True,le=True,npr=True,amh=True,tsc=True,wa=True,pl5=True,dp=True,slr=True,lower_d=False)
    features,label=hr_preprocessing(dp=True)
    #print(hr_preprocessing())
    #print(hr_processing(sl=True,le=True,npr=True,dp=True,lower_d=True,ld_n=3))
    hr_modeling(features,label)
    
if __name__=='__main__':
    main()

8999 3000 3000
Train：
ACC: 0.9736637404156018
REC: 0.9595160539785947
F-SCORE: 0.9456546663609264
Validation：
ACC: 0.9453333333333334
REC: 0.9280677009873061
F-SCORE: 0.8891891891891892
ACC: 0.9453333333333334
REC: 0.9280677009873061
F-SCORE: 0.8891891891891892
Test:
ACC: 0.9406666666666667
REC: 0.8934081346423562
F-SCORE: 0.8774104683195592
Test2:
ACC: 0.9406666666666667
REC: 0.8934081346423562
F-SCORE: 0.8774104683195592


In [ ]:
#经测试neighbors=3时效果大于5

##模型改造


In [25]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
#sl:satisfaction_level-False:MinMaxScaler,True:StandardScaler
#le:last_evaluation-False:MinMaxScaler,True:StandardScaler
#npr:number_project-False;MinMaxScaler,True:StandardScaler
#amh:average_monthly-hours-False;MinMaxScaler,True:StandardScaler
#tsc:time_spend_company-False;MinMaxScaler,True:StandardScaler
#wa:work_accident-False;MinMaxScaler,True:StandardScaler
#pl5:promotion_last_5years-False;MinMaxScaler,True:StandardScaler
#dp:department-False:LabelEncoding,True:OneHotEncoding
#slr；salary-False:LabelEncoding,True:OneHotEncoding
#lower_d指定是否需要降维，如需要默认降到一维
def hr_preprocessing(sl=False,le=False,npr=False,amh=False,tsc=False,wa=False,pl5=False,dp=False,slr=False,lower_d=False,ld_n=1):
    df=pd.read_csv('./data/HR.csv')
   
    #1.清洗数据(去除异常值、抽样)
    df=df.dropna(subset=['satisfaction_level','last_evaluation'])
    df=df[df['satisfaction_level']<=1][df['salary']!='nme']
     #2。得到标注
    label = df['left']
    df=df.drop('left',axis=1)#axis=1指定以列删除
    #3.特征选择
    #4.特征处理  
    scaler_lst=[sl,le,npr,amh,tsc,wa,pl5]
    column_lst=['satisfaction_level','last_evaluation','number_project','average_monthly_hours','time_spend_company','Work_accident','promotion_last_5years']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=\
                MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]#二维格式
        else:
            df[column_lst[i]]=\
                StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
      
    scaler_lst=[dp,slr]
    column_lst=['department','salary']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i]=='salary':
                df[column_lst[i]]=[map_salary(s) for s in df['salary'].values]
            else:
                df[column_lst[i]]=LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df=pd.get_dummies(df,columns=[column_lst[i]])
    if lower_d:
        return PCA(n_components=ld_n).fit_transform(df,values).label#PVA降维，可不使用标注
    return df,label
#使salary返回对应的等级而不是012
d=dict([('low',0),('medium',1),('high',2)])
def map_salary(s):
    return d.get(s,0)
#建模
def hr_modeling(features,label):
    from sklearn.model_selection import train_test_split  #切分训练集测试集
    f_v=features.values
    l_v=label.values
    X_tt,X_validation,Y_tt,Y_validation=train_test_split(f_v,l_v,test_size=0.2)   #获得验证集占20%
    X_train,X_test,Y_train,Y_test=train_test_split(X_tt,Y_tt,test_size=0.25)  #训练集、测试集
    #print(len(X_train),len(X_validation),len(X_test))#得到6：2：2
    #KNN
    from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier
    from sklearn.metrics import accuracy_score,recall_score,f1_score#F一
    models=[]
    models.append(('KNN',KNeighborsClassifier(n_neighbors=3)))
    for clf_name,clf in models:
        clf.fit(X_train,Y_train)
        xy_lst=[(X_train,Y_train),(X_validation,Y_validation),(X_test,Y_test)]#依次用0，1，2表示
        for i in range(len(xy_lst)):
            X_part=xy_lst[i][0]
            Y_part=xy_lst[i][1]
            Y_pred=clf.predict(X_part)
            print(i)
            print(clf_name,'-ACC',accuracy_score(Y_part,Y_pred))
            print(clf_name,'-REC',recall_score(Y_part,Y_pred))
            print(clf_name,'-F1',f1_score(Y_part,Y_pred))
        

def main():
    #features,label=hr_preprocessing(sl=True,le=True,npr=True,amh=True,tsc=True,wa=True,pl5=True,dp=True,slr=False,lower_d=False)
    #features,label=hr_preprocessing(sl=True,le=True,npr=True,amh=True,tsc=True,wa=True,pl5=True,dp=True,slr=True,lower_d=False)
    features,label=hr_preprocessing(dp=True)
    #print(hr_preprocessing())
    #print(hr_processing(sl=True,le=True,npr=True,dp=True,lower_d=True,ld_n=3))
    hr_modeling(features,label)
    
if __name__=='__main__':
    main()

0
KNN -ACC 0.969552172463607
KNN -REC 0.9523141654978962
KNN -F1 0.9369825206991719
1
KNN -ACC 0.9483333333333334
KNN -REC 0.9067909454061251
KNN -F1 0.8978246539222149
2
KNN -ACC 0.942
KNN -REC 0.8913362701908958
KNN -F1 0.8746397694524496
